In [1]:
import os
#Legacy TensorFlow BackEnd
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [15]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pathlib
import tensorflow_text as tf_text
import pickle

In [3]:
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Number of devices: 2


I0000 00:00:1766163538.204091      47 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1766163538.204693      47 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [4]:
config = {
    "learning_rate": 1e-4,
    "batch_size": 256,
    "epochs": 20,
    "max_vocab_size":5000,
    "max_length":50
}

GLOBAL_BATCH = config['batch_size'] * strategy.num_replicas_in_sync
LEARNING_RATE = config['learning_rate']
EPOCHS = config['epochs']

In [5]:
path_to_zip = tf.keras.utils.get_file(
    'spa-eng.zip', origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
    extract=True)

path_to_file = pathlib.Path(path_to_zip).parent/'spa-eng/spa.txt'

2638744/2638744 [==============================] - 0s 0us/step


In [6]:
def load_data(path):
  text = path.read_text(encoding='utf-8')

  lines = text.splitlines()
  pairs = [line.split('\t') for line in lines]

  context = np.array([context for target, context in pairs])
  target = np.array([target for target, context in pairs])

  return target, context

In [7]:
context_raw,target_raw = load_data(path_to_file)
print(context_raw[-1])

If you want to sound like a native speaker, you must be willing to practice saying the same sentence over and over in the same way that banjo players practice the same phrase over and over until they can play it correctly and at the desired tempo.


In [8]:
print(target_raw[-1])

Si quieres sonar como un hablante nativo, debes estar dispuesto a practicar diciendo la misma frase una y otra vez de la misma manera en que un músico de banjo practica el mismo fraseo una y otra vez hasta que lo puedan tocar correctamente y en el tiempo esperado.


In [9]:
split_idx = int(0.9 * len(target_raw))

X_train = context_raw[:split_idx]
y_train = target_raw[:split_idx]

X_val = context_raw[split_idx:]
y_val = target_raw[split_idx:]

BUFFER_SIZE = len(X_train)

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(GLOBAL_BATCH, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

# Validation dataset (no shuffle needed)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_dataset = val_dataset.batch(GLOBAL_BATCH, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

In [10]:
idx = np.random.randint(0,63)
context,target = next(iter(val_dataset))
print(f"Input:{context[idx].numpy().decode('utf-8')}")
print(f"Target:{target[idx].numpy().decode('utf-8')}")

Input:Tom celebrated his twentieth birthday last week.
Target:Tom celebró su vigésimo cumpleaños la semana pasada.


In [11]:
def tf_lower_and_split_punct_w_special_tokens(text):
  text = tf_text.normalize_utf8(text, 'NFKD')
  text = tf.strings.lower(text)
  text = tf.strings.regex_replace(text, '[^ a-z.?!,¿]', '')
  text = tf.strings.regex_replace(text, '[.?!,¿]', r' \0 ')
  text = tf.strings.strip(text)

  text = tf.strings.join(['[START]', text, '[END]'], separator=' ')
  return text

def tf_lower_and_split_punct(text):
  text = tf_text.normalize_utf8(text, 'NFKD')
  text = tf.strings.lower(text)
  text = tf.strings.regex_replace(text, '[^ a-z.?!,¿]', '')
  text = tf.strings.regex_replace(text, '[.?!,¿]', r' \0 ')
  text = tf.strings.strip(text)

  return text

In [12]:
print(target[idx].numpy().decode())
print(tf_lower_and_split_punct(target[idx]).numpy().decode())
print(tf_lower_and_split_punct_w_special_tokens(target[idx]).numpy().decode())

Tom celebró su vigésimo cumpleaños la semana pasada.
tom celebro su vigesimo cumpleanos la semana pasada .
[START] tom celebro su vigesimo cumpleanos la semana pasada . [END]


In [16]:
with open('/kaggle/input/vocab/tensorflow2/default/1/context_vocab.pkl', 'rb') as f:
    context_vocab = pickle.load(f)

with open('/kaggle/input/vocab/tensorflow2/default/1/target_vocab.pkl', 'rb') as f:
    target_vocab = pickle.load(f)

In [17]:
context_text_processor = tf.keras.layers.TextVectorization(
    standardize = tf_lower_and_split_punct,
    max_tokens = config['max_vocab_size'],
    output_sequence_length = config['max_length'],
    vocabulary = context_vocab,
    ragged = False)

target_text_processor = tf.keras.layers.TextVectorization(
    standardize = tf_lower_and_split_punct_w_special_tokens,
    max_tokens = config['max_vocab_size'],
    output_sequence_length = config['max_length'] + 1,
    vocabulary = target_vocab,
    ragged = False)

In [18]:
# context_text_processor.adapt(train_dataset.map(lambda context, target: context))

print(context_text_processor.get_vocabulary()[:10])


# target_text_processor.adapt(train_dataset.map(lambda context, target: target))
target_text_processor.get_vocabulary()[:10]

['', '[UNK]', '.', 'i', 'the', 'to', 'you', 'tom', '?', 'a']


['', '[UNK]', '[START]', '[END]', '.', 'que', 'el', 'de', 'no', 'tom']

In [19]:
def process_text(context, target):
  context = context_text_processor(context)
  target = target_text_processor(target)
  targ_in = target[:,:-1]
  targ_out = target[:,1:]
  return (context, targ_in), targ_out


train_dataset = train_dataset.map(process_text, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.map(process_text, num_parallel_calls = tf.data.AUTOTUNE)

In [20]:
for (ex_context_tok, ex_tar_in), ex_tar_out in val_dataset.take(1):
  print(ex_context_tok[0, :10].numpy()) 
  print(ex_context_tok.shape)
  print(ex_tar_in[0, :10].numpy()) 
  print(ex_tar_out[0, :10].numpy())
  print(ex_tar_out.shape)

[  18   10    4  297 4408   19    3  212  179    2]
(512, 50)
[   2   42 4238   14 1118   36    5  592  137    4]
[  42 4238   14 1118   36    5  592  137    4    3]
(512, 50)


In [31]:
def Build_Seq2Seq(max_length=157, vocab_size_en=10000, vocab_size_es=10000, embedding_dim=256, units=512):
    encoder_input = tf.keras.layers.Input(shape=(max_length,), dtype="int32", name='encoder_input')
    
    enc_emb = tf.keras.layers.Embedding(vocab_size_en, embedding_dim, mask_zero=True)(encoder_input)
    enc_emb = tf.keras.layers.Dropout(0.2)(enc_emb)
    
    encoder = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(
            units, 
            return_sequences=True, 
            return_state=True,
            name='encoder_lstm'
        )
    )
    
    encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder(enc_emb)
    
    encoder_state_h = tf.keras.layers.Concatenate(axis=-1)([forward_h, backward_h])
    encoder_state_c = tf.keras.layers.Concatenate(axis=-1)([forward_c, backward_c])
    
    decoder_input = tf.keras.layers.Input(shape=(max_length,), dtype="int32", name='decoder_input')
    
    dec_emb = tf.keras.layers.Embedding(vocab_size_es, embedding_dim, mask_zero=True)(decoder_input)
    dec_emb = tf.keras.layers.Dropout(0.2)(dec_emb)
    
    decoder_outputs = tf.keras.layers.LSTM(
        units * 2,
        return_sequences=True,
        return_state=False,
        name='decoder_lstm'
    )(dec_emb, initial_state=[encoder_state_h, encoder_state_c])
    
   
    attention_output = tf.keras.layers.MultiHeadAttention(
        num_heads=8,
        key_dim=units * 2, 
        name='cross_attention'
    )(
        query=decoder_outputs, 
        value=encoder_outputs,  
        key=encoder_outputs     
    )
    
    decoder_combined = tf.keras.layers.Concatenate(axis=-1)([decoder_outputs, attention_output])
    
    outputs = tf.keras.layers.Dense(vocab_size_es, activation='softmax', name='output_dense')(decoder_combined)
    
    model = tf.keras.Model(inputs=[encoder_input, decoder_input], outputs=outputs, name='seq2seq')
    return model
vocab_size_en = context_text_processor.vocabulary_size()
vocab_size_fr = target_text_processor.vocabulary_size()
print(f"English vocab size: {vocab_size_en}")
print(f"French vocab size: {vocab_size_fr}")

with strategy.scope():
    model = Build_Seq2Seq(
        max_length=config['max_length'], 
        vocab_size_en=vocab_size_en,
        vocab_size_es=vocab_size_fr,
        embedding_dim=256,
        units=256  
      
)

English vocab size: 5000
French vocab size: 5000


In [32]:
model.summary()

Model: "seq2seq"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 encoder_input (InputLayer)  [(None, 50)]                 0         []                            
                                                                                                  
 embedding_4 (Embedding)     (None, 50, 256)              1280000   ['encoder_input[0][0]']       
                                                                                                  
 decoder_input (InputLayer)  [(None, 50)]                 0         []                            
                                                                                                  
 dropout_4 (Dropout)         (None, 50, 256)              0         ['embedding_4[0][0]']         
                                                                                            

In [33]:
def masked_loss(y_true, y_pred):
    
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=False,
        reduction='none'
    )
    loss = loss_fn(y_true, y_pred) 

    mask = tf.cast(y_true != 0, loss.dtype)
    loss *= mask

    return tf.reduce_sum(loss) / tf.reduce_sum(mask)


def masked_acc(y_true, y_pred):
    
    y_pred = tf.argmax(y_pred, axis=-1)
    y_pred = tf.cast(y_pred, y_true.dtype)

    match = tf.cast(y_true == y_pred, tf.float32)
    
    mask = tf.cast(y_true != 0, tf.float32)

    return tf.reduce_sum(match) / tf.reduce_sum(mask)

In [34]:
with strategy.scope():
    model.compile(optimizer='adam',
                  loss=masked_loss, 
                  metrics=[masked_acc])

In [35]:
history = model.fit(
    train_dataset, 
    epochs=30,
    validation_data=val_dataset,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.1,          
            patience=3,           
            min_lr=1e-7,          
            verbose=1             
        )
    ]
)

Epoch 1/30
INFO:tensorflow:Collective all_reduce tensors: 19 all_reduces, num_devices = 2, group_size = 2, implementation = CommunicationImplementation.NCCL, num_packs = 1
INFO:tensorflow:Collective all_reduce IndexedSlices: 2 all_reduces, num_devices =2, group_size = 2, implementation = CommunicationImplementation.NCCL
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Collective all_reduce tensors: 19 all_reduces, num_devices = 2, group_size = 2, implementation = Commu

I0000 00:00:1766163741.123267     115 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1766163741.123282     113 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1766163743.313329     113 service.cc:148] XLA service 0x7e1c92a1d690 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1766163743.314329     113 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1766163743.314356     113 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1766163743.554911     115 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


209/209 [==============================] - ETA: 0s - loss: 4.6367 - masked_acc: 0.2823INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
209/209 [==============================] - 125s 485ms/step - loss: 4.6367 - masked_acc: 0.2823 - val_loss: 4.4665 - val_masked_acc: 0.2676 - lr: 0.0010
Epoch 2/30
209/209 [==============================] - 92s 437ms/step - loss: 2.8462 - masked_acc: 0.4952 - val_loss: 3.1098 - val_masked_acc: 0.4372 - lr: 0.0010
Epoch 3/30
209/209 [==============================] - 90s 431ms/step - loss: 1.8136 - masked_acc: 0.6432 - val_loss: 2.4368 - val_masked_acc: 0.5293 - lr: 0.0010
Epoch 4/30
209/209 [==============================] - 89s 426ms/step - loss: 1.3400 - masked_acc: 0.7142 - val_loss: 2.1041 - val_masked_acc: 0.5756 - lr: 0.0

In [36]:
model.save("BI-LSTM-Cross-ATT.keras")

In [37]:
model.save_weights("BI-LSTM-Cross-ATT.h5")

In [38]:
loaded_model = tf.keras.models.load_model("/kaggle/working/BI-LSTM-Cross-ATT.keras",compile=False)

In [39]:
(ex_context_tok, ex_tar_in), ex_tar_out = next(iter(val_dataset))
print(ex_context_tok.shape)
print(ex_tar_in[0, :10].numpy()) 
print(ex_tar_out[0, :10].numpy())
print(ex_tar_out.shape)

(512, 50)
[   2   42 4238   14 1118   36    5  592  137    4]
[  42 4238   14 1118   36    5  592  137    4    3]
(512, 50)


In [40]:
def get_initial_state(model, context, target_text_processor):
    if len(context.shape) == 1:
        context = tf.expand_dims(context, 0)
    
    batch_size = tf.shape(context)[0]
    vocab = target_text_processor.get_vocabulary()
    start_token = vocab.index('[START]')
    
    next_token = tf.fill([batch_size, 1], start_token)
    done = tf.zeros([batch_size, 1], dtype=tf.bool)
    
    return next_token, done, context


def get_next_token(model, context, next_token, done, state, target_text_processor, temperature=0.0):
    vocab = target_text_processor.get_vocabulary()
    end_token = vocab.index('[END]')
    
    padded_state = tf.pad(state, [[0, 0], [0, config['max_length'] - tf.shape(state)[1]]])[:, :config['max_length']]
    logits = model.predict([context, padded_state], verbose=0)
    last_logits = logits[:, tf.shape(state)[1] - 1, :]
    
    if temperature == 0.0:
        next_token_id = tf.argmax(last_logits, axis=-1, output_type=tf.int32)
    else:
        next_token_id = tf.squeeze(tf.random.categorical(last_logits / temperature, 1, dtype=tf.int32), -1)
    
    next_token_id = tf.expand_dims(next_token_id, -1)
    done = done | (next_token_id == end_token)
    next_token_id = tf.where(done, tf.constant(0, dtype=tf.int32), next_token_id)
    new_state = tf.concat([state, next_token_id], axis=1)
    
    return next_token_id, done, new_state


def translate(model, spanish_text, target_text_processor, temperature=0.0):
    if len(spanish_text.shape) == 1:
        spanish_text = tf.expand_dims(spanish_text, 0)
    
    next_token, done, context = get_initial_state(model, spanish_text, target_text_processor)
    state = next_token
    tokens = []
    
    for n in range(config['max_length']):
        next_token, done, state = get_next_token(model, context, next_token, done, state, target_text_processor, temperature)
        tokens.append(next_token)
        if tf.reduce_all(done):
            break
    
    tokens = tf.concat(tokens, axis=-1)
    vocab = target_text_processor.get_vocabulary()
    
    words = []
    for token_id in tokens[0].numpy():
        if token_id == 0:
            break
        word = vocab[token_id]
        if word == '[END]':
            break
        if word not in ['[START]', '[UNK]', '']:
            words.append(word)
    
    return ' '.join(words)


def compare_translations(model, spanish_input, target_out, context_text_processor, target_text_processor, n=5):
    spanish_vocab = context_text_processor.get_vocabulary()
    english_vocab = target_text_processor.get_vocabulary()
    
    for i in range(min(n, spanish_input.shape[0])):
        # Spanish input
        sp_words = [spanish_vocab[t] for t in spanish_input[i].numpy() if t > 0]
        spanish = ' '.join(sp_words)
        
        # Ground truth English
        gt_words = [english_vocab[t] for t in target_out[i].numpy() 
                    if t > 0 and english_vocab[t] not in ['[START]', '[END]']]
        ground_truth = ' '.join(gt_words)
        
        # Model translation
        model_output = translate(model, spanish_input[i], target_text_processor)
        
        print(f"\n{i+1}. English: {spanish}")
        print(f"   GROUNDTRUTH: {ground_truth}")
        print(f"   TRANSLATION: {model_output}")


val_iter = iter(val_dataset)

(ex_context_tok, ex_tar_in), ex_tar_out = next(val_iter)
compare_translations(loaded_model, ex_context_tok, ex_tar_out, context_text_processor, target_text_processor, n=5)

(ex_context_tok, ex_tar_in), ex_tar_out = next(val_iter)
compare_translations(loaded_model, ex_context_tok, ex_tar_out, context_text_processor, target_text_processor, n=5)

(ex_context_tok, ex_tar_in), ex_tar_out = next(val_iter)
compare_translations(loaded_model, ex_context_tok, ex_tar_out, context_text_processor, target_text_processor, n=5)


1. English: this is the same necklace that i lost yesterday .
   GROUNDTRUTH: este collar es igual al que perdi ayer .
   TRANSLATION: este es el mismo jugador que perdi ayer .

2. English: this is the strongest dog that i have ever seen .
   GROUNDTRUTH: este es el perro mas fuerte que jamas haya visto .
   TRANSLATION: este es el perro mas fuerte que haya visto nunca .

3. English: this is your last chance to spend time with tom .
   GROUNDTRUTH: esta es tu ultima oportunidad de pasar tiempo con tom .
   TRANSLATION: esta es tu ultima oportunidad de pasar tiempo con tom .

4. English: this material will stand up to lots of [UNK] .
   GROUNDTRUTH: este material [UNK] un monton de [UNK] .
   TRANSLATION: este se a muchas .

5. English: this medicine should be taken every three hours .
   GROUNDTRUTH: este medicamento debe ser tomado cada tres horas .
   TRANSLATION: este remedio deberia cada tres horas .

1. English: his salary is double what it was seven years ago .
   GROUNDTRUTH: s